1. Попытаться ответить на вопросы/выдвинуть гипотезы

- А точно нужно сортировать по вероятности?<br>
Думаю, что сортировать по вероятности необходимо, чтобы рекомендовать наиболее подходящие товары<br><br>
- Какую метрику использовать?<br>
Если исходить из задачи бизнеса, сформиулированную как "увеличение продаж" в виде увеличения прибыли, что надо использовать метрику Money Precision@k. Если же "увеличение продаж" как количество покупок, то для e-mail рассылки я бы использовал метрику Precision@k, если на следующей итерации покупок юзером можно было оценить качество рекомендации. То для печати рекомендаций на чеке у кассы (офлайн) я бы использовал метрику Recall@k, чтобы оценивать качество модели рекомендаций.<br><br>
- Сколько раз в неделю отправляем рассылку?<br>
Я думаю, что оптимальным было бы отправлять либо в пятницу вечером для клиентов гипермаркетов перед походом в магазин на выходных на будущую неделю. Либо во время проведения акций.<br><br>
- В какое время отправляем рассылку?<br>
Рассылку отправляем либо в обеденное, либо в вечернее время 17-20:00.<br><br>
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?<br>
Могу только предположить, что можно вести список ранее рекомендованных товаров. Но не знаю, насколько это оправданно в реале.<br><br>
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?<br>
Думаю, что товары точно должным быть разными, топы из своих категорий, чтобы охватить как можно больше возможных потребностей.

4. Поэкспериментировать с ALS (grid-search) ********
   * Подобрать лучшие параметры
   * Написать отчет об увиденном

In [1]:
# !pip install implicit==0.4.4

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)
  
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [4]:
data = pd.read_csv('../data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [5]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

C:\Users\iGrom\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# ALS

In [10]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [11]:
als_regularization = [0.05, 0.1]
als_iterations = [5, 10, 15]

result_dict = {'metric':0, 'factors':0, 'regularization':0, 'iterations':0}

In [12]:
%%time

for reg in als_regularization:
    for iter in als_iterations:
        model = AlternatingLeastSquares(factors=64, 
                                  regularization=reg,
                                  iterations=iter, 
                                  calculate_training_loss=True, 
                                  num_threads=10,
                                  use_gpu=False)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        metric = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
        result.drop(labels='als', axis=1)

        if metric >= result_dict['metric']:
            result_dict['metric'] = metric
            result_dict['factors'] = 64
            result_dict['regularization'] = reg
            result_dict['iterations'] = iter

result_dict


Wall time: 7min 10s


{'metric': 0.1669512807634331,
 'factors': 64,
 'regularization': 0.05,
 'iterations': 5}

In [13]:
als_factors = [32, 64, 128, 256]
for fact in als_factors:
    model = AlternatingLeastSquares(factors=fact, 
                              regularization=0.05,
                              iterations=5, 
                              calculate_training_loss=True, 
                              num_threads=10,
                              use_gpu=False)
    model.fit(csr_matrix(user_item_matrix).T.tocsr())
    result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    metric = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    result.drop(labels='als', axis=1)

    if metric >= result_dict['metric']:
        result_dict['metric'] = metric
        result_dict['factors'] = fact

result_dict

{'metric': 0.1669512807634331,
 'factors': 64,
 'regularization': 0.05,
 'iterations': 5}

### 2. TF-IDF взвешивание

In [14]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T

In [15]:
%%time

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=10)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 3.69 s


In [16]:
result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

0.1697639377197364

### 3. BM25 взвешивание

In [17]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\iGrom\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [20]:
%%time

# model = AlternatingLeastSquares(factors=128, 
#                                 regularization=0.05,
#                                 iterations=15, 
#                                 calculate_training_loss=True, 
#                                 num_threads=4) # K - кол-во билжайших соседей

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=10)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))


Wall time: 1min 8s


In [21]:
result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

0.18151682571571798

Подбирая гиперпараметры для обучения модели увидел, что оптимальным является число факторов равное 64, несмотря на то, что при проверки были и большие значения. А вот значение регуляризации и число итераций получилось минимальное из проверяемых значений: 0.05 и 5 соответственно. Значительное улучшение метрики дало применение взвешивания BM25.